In [1]:
import os

import numpy as np
import pandas as pd
import us
from census import Census
from dotenv import load_dotenv
from scipy import stats as st

from hxpr.utils import here

load_dotenv()

True

In [2]:
DATA_DIR = here() / "data" / "phase2"
INPUT_DATA_DIR = DATA_DIR / "original"
OUTPUT_DATA_DIR = DATA_DIR / "merged"
EXPLODED_DATA_DIR = DATA_DIR / "exploded"

for ddir in [INPUT_DATA_DIR, OUTPUT_DATA_DIR, EXPLODED_DATA_DIR]:
    ddir.mkdir(parents=True, exist_ok=True)

BY_DEMOGRAPHICS_DATA = INPUT_DATA_DIR / "DePaul Campaign Data - Demographics.xlsx"
BY_STATE_DATA = INPUT_DATA_DIR / "DePaul Campaign Data - State.xlsx"
BY_ZIP_CODE_DATA = INPUT_DATA_DIR / "Depaul Campaign Data - Zip Codes.xlsx"
BY_NOTHING_DATA = (
    INPUT_DATA_DIR / "DePaul Campaign Data - Campaign & Ad Performance.xlsx"
)

In [3]:
dfs = pd.read_excel(BY_ZIP_CODE_DATA, sheet_name=None, skiprows=1)
to_concat = []
for df in dfs.values():
    df = df.copy()
    df = df.drop(columns=[col for col in df if col.startswith("Unnamed")])

    df["zipcode"] = df["Matched location"].str[:5]
    df = df.rename(
        columns={
            "Clicks": "clicks",
            "Impr.": "impressions",
            "CTR": "ctr",
            "Avg. CPC": "average_cpc",
            "Cost": "cost",
            "Campaign": "campaign",
        }
    )

    df["campaign_name"] = df["campaign"].str[:-3]
    df["language"] = df["campaign"].str[-2:].str.lower()
    df["cost"] = df["cost"].apply(lambda x: float("nan") if isinstance(x, str) else x)
    df["average_cpc"] = df["average_cpc"].apply(
        lambda x: float("nan") if isinstance(x, str) else x
    )
    df = df[
        [
            "campaign_name",
            "language",
            "zipcode",
            "clicks",
            "impressions",
            "ctr",
            "cost",
            "average_cpc",
            "campaign",
        ]
    ]
    to_concat.append(df)

final_zip_df = pd.concat(to_concat)
final_zip_df.to_parquet(OUTPUT_DATA_DIR / "data_by_zipcode.parquet")

In [4]:
dfs = pd.read_excel(BY_DEMOGRAPHICS_DATA, sheet_name=None, skiprows=1)
to_concat = []
for df in dfs.values():
    df = df.copy()
    df = df.drop(columns=[col for col in df if col.startswith("Unnamed")])

    df = df.rename(
        columns={
            "Age": "age",
            "Gender": "gender",
            "Parental status": "is_a_parent",
            "Household income": "household_income",
            "CTR": "ctr",
            "Avg. CPC": "average_cpc",
            "Cost": "cost",
            "Campaign": "campaign",
            "Clicks": "clicks",
            "Impr.": "impressions",
        }
    )

    df["campaign_name"] = df["campaign"].str[:-3]
    df["language"] = df["campaign"].str[-2:].str.lower()
    df["cost"] = df["cost"].apply(lambda x: float("nan") if isinstance(x, str) else x)
    df["average_cpc"] = df["average_cpc"].apply(
        lambda x: float("nan") if isinstance(x, str) else x
    )
    df = df[
        [
            "campaign_name",
            "language",
            "age",
            "gender",
            "is_a_parent",
            "household_income",
            "clicks",
            "impressions",
            "ctr",
            "cost",
            "average_cpc",
            "campaign",
        ]
    ]
    assert df["is_a_parent"].isin(["Parent", "Not a parent"]).all()
    df["is_a_parent"] = df["is_a_parent"] != "Not a parent"
    to_concat.append(df)

final_demo_df = pd.concat(to_concat)
final_demo_df.to_parquet(OUTPUT_DATA_DIR / "data_by_demographics.parquet")

In [5]:
dfs = pd.read_excel(BY_STATE_DATA, sheet_name=None, skiprows=1)
to_concat = []
for df in dfs.values():
    df = df.copy()
    df = df.drop(columns=[col for col in df if col.startswith("Unnamed")])

    df = df.rename(
        columns={
            "CTR": "ctr",
            "Avg. CPC": "average_cpc",
            "Cost": "cost",
            "Campaign": "campaign",
            "Clicks": "clicks",
            "Impr.": "impressions",
        }
    )
    df["state"] = df["Location"].str.split(",").apply(lambda x: x[0])
    df["state_abbr"] = df["state"].apply(lambda x: us.states.lookup(x).abbr)
    df["state_fips"] = df["state"].apply(lambda x: us.states.lookup(x).fips)

    df["campaign_name"] = df["campaign"].str[:-3]
    df["language"] = df["campaign"].str[-2:].str.lower()
    df["cost"] = df["cost"].apply(lambda x: float("nan") if isinstance(x, str) else x)
    df["average_cpc"] = df["average_cpc"].apply(
        lambda x: float("nan") if isinstance(x, str) else x
    )
    df = df[
        [
            "campaign_name",
            "language",
            "state",
            "state_abbr",
            "state_fips",
            "clicks",
            "impressions",
            "ctr",
            "cost",
            "average_cpc",
            "campaign",
        ]
    ]
    to_concat.append(df)

final_state_df = pd.concat(to_concat)
assert len(final_state_df["state_fips"].value_counts(dropna=False).value_counts())
final_state_df.to_parquet(OUTPUT_DATA_DIR / "data_by_state.parquet")

In [6]:
missing_state_fips = {state.fips for state in us.STATES} - set(df["state_fips"])
for msf in missing_state_fips:
    print(us.states.lookup(msf).name)

Maryland
Rhode Island


In [7]:
final_state_df["impressions"].sum()

5616715

In [8]:
final_demo_df["impressions"].sum()

5616715

In [9]:
final_zip_df["impressions"].sum()

3466884

### Some Notes

* Final impression counts in state and demo totals are the same, but slightly _above_ the summary "Campaign Performance sheet
* Final impressinon counts from the ZIP code tables are 60% lower
* The data does not contain details on the individual combinations that were put together in each location. However, they do have a description in final sheet (not described here) of "Low, Good, Best" for the individual asset combinations

## Appending Census Data

### By ZIP Code (ZCTA?)

In [10]:
c = Census(os.environ["CENSUS_API_KEY"])

In [11]:
zcta_df = pd.DataFrame(
    c.acs5.zipcode(["B01001_001E"], zcta="*", state_fips="*", year=2019)
)
zcta_df = zcta_df.rename(
    columns={
        "B01001_001E": "total_population",
        "state": "state_fips",
        "zip code tabulation area": "zipcode",
    }
)
merged_zip_df = final_zip_df.merge(zcta_df, how="left", on="zipcode")
merged_zip_df.to_parquet(OUTPUT_DATA_DIR / "data_by_zipcode_merged_census.parquet")
merged_zip_df[merged_zip_df["state_fips"].isna()]

,campaign_name,language,zipcode,clicks,impressions,ctr,cost,average_cpc,campaign,total_population,state_fips
1118,Self-Oriented,en,06338,0,7,0.0,0.0,NaN,Self-Oriented EN,NaN,NaN
1935,Self-Oriented,en,11249,0,3,0.0,0.0,NaN,Self-Oriented EN,NaN,NaN
3717,Self-Oriented,en,18711,0,12,0.0,0.0,NaN,Self-Oriented EN,NaN,NaN
4127,Self-Oriented,en,22649,0,1,0.0,0.0,NaN,Self-Oriented EN,NaN,NaN
4408,Self-Oriented,en,24061,0,21,0.0,0.0,NaN,Self-Oriented EN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
139515,Personal Responsibility,sp,92093,0,4,0.0,0.0,NaN,Personal Responsibility SP,NaN,NaN
139548,Personal Responsibility,sp,92179,0,7,0.0,0.0,NaN,Personal Responsibility SP,NaN,NaN
139928,Personal Responsibility,sp,93737,0,4,0.0,0.0,NaN,Personal Responsibility SP,NaN,NaN
140174,Personal Responsibility,sp,95141,0,2,0.0,0.0,NaN,Personal Responsibility SP,NaN,NaN


### More notes

So that first ZIP code (06338) is a [P.O. Box ZIP Code](https://www.zip-codes.com/zip-code/06338/zip-code-06338.asp), which strikes me as odd. What exactly is going on with this ZIP code data. (Note that that website is usually accurate, though you should really double check with SmartyStreets, which has the raw USPS data.)

In [12]:
table = final_state_df.groupby("language")["clicks", "impressions"].sum()
print(table["clicks"] / table["impressions"])
table["impressions"] = table["impressions"] - table["clicks"]
st.contingency.chi2_contingency(table.values.T)

language
en    0.011085
sp    0.013317
dtype: float64


/var/folders/_8/wxdcgmrd7b931622mzvpxn6c0000gq/T/ipykernel_57977/4170725730.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  table = final_state_df.groupby("language")["clicks", "impressions"].sum()


(580.8663117592343,
 2.430191684357767e-128,
 1,
 array([[  36201.79854363,   31935.20145637],
        [2948009.20145637, 2600568.79854363]]))

In [13]:
table = (
    final_state_df[final_state_df["language"] == "en"]
    .groupby(["campaign"])[["clicks", "impressions"]]
    .sum()
)
print(table["clicks"] / table["impressions"])

campaign
Helping Community EN          0.011268
Helping Others EN             0.010803
Personal Responsibility EN    0.012852
Self-Oriented EN              0.009820
dtype: float64


In [14]:
table = (
    final_state_df[final_state_df["language"] == "sp"]
    .groupby(["campaign"])[["clicks", "impressions"]]
    .sum()
)
print(table["clicks"] / table["impressions"])

campaign
Helping Community SP          0.016799
Helping Others SP             0.012042
Personal Responsibility SP    0.011959
Self-Oriented SP              0.013431
dtype: float64


### Explode data

Hansoo requests the data be broken into one line per impression. This is done below.

In [15]:
for filename in OUTPUT_DATA_DIR.glob("data*.parquet"):
    print(f"Exploding {filename.name}...")
    df = pd.read_parquet(filename)
    df["num_in_group"] = [
        np.arange(num_impressions, dtype=int)
        for num_impressions in df["impressions"].values
    ]
    exploded_df = df.explode("num_in_group")
    exploded_df["was_clicked"] = exploded_df["num_in_group"] < exploded_df["clicks"]
    exploded_df = exploded_df.drop(
        columns=["clicks", "impressions", "ctr", "cost", "average_cpc", "num_in_group"]
    )
    exploded_df.to_parquet(EXPLODED_DATA_DIR / f"exploded_{filename.name}")
    exploded_df.to_csv(
        EXPLODED_DATA_DIR / f"exploded_{filename.with_suffix('').name}.csv.gz",
        index=False,
    )
print("Done")

Exploding data_by_state.parquet...
Exploding data_by_demographics.parquet...
Exploding data_by_zipcode_merged_census.parquet...
Exploding data_by_zipcode.parquet...
Done
